
# Bayesian Optimization with Retrieval Optimizer

This notebook demonstrates how to use Bayesian optimization to tune Redis-based retrieval pipelines. Unlike a grid study—which tests all combinations—Bayesian optimization intelligently searches the configuration space, prioritizing promising settings based on previous results. This is especially useful when the number of possible configurations is large and exhaustive search would be too costly.

You'll define a study configuration, specify embedding models and search methods, and let the optimizer guide the search toward the best-performing retrieval setup.


## Dataset

We'll import a dataset from the [beir benchmark IR project](https://github.com/beir-cellar/beir) to get going quickly. 



In [ ]:
# Load data
from redis_retrieval_optimizer.corpus_processors import eval_beir

# check the link above for different datasets to try
beir_dataset_name = "nfcorpus"

# Load sample data
corpus, queries, qrels = eval_beir.get_beir_dataset(beir_dataset_name)

## Study config

In this directory there is a yaml file containing a configuration for a bayesian study that looks like this:

```yaml
# path to data files for easy read
corpus: "data/nfcorpus_corpus.json"
queries: "data/nfcorpus_queries.json"
qrels: "data/nfcorpus_qrels.json"

index_settings:
  name: "optimize"
  vector_field_name: "vector" # name of the vector field to search on
  text_field_name: "text" # name of the text field for lexical search
  from_existing: false
  vector_dim: 384 # should match first embedding model or from_existing
  additional_fields:
      - name: "title"
        type: "text"

optimization_settings:
  # defines the options optimization can take
  metric_weights:
    f1_at_k: 1
    embedding_latency: 1
    total_indexing_time: 1
  algorithms: ["hnsw"]
  vector_data_types: ["float16", "float32"]
  distance_metrics: ["cosine"]
  n_trials: 10
  n_jobs: 1
  ret_k: [1, 10] # potential range of value to be sampled during study
  ef_runtime: [10, 20, 30, 50]
  ef_construction: [100, 150, 200, 250, 300]
  m: [8, 16, 64]


search_methods: ["vector", "lin_combo"]
embedding_models:
  - type: "hf"
    model: "sentence-transformers/all-MiniLM-L6-v2"
    dim: 384
    embedding_cache_name: "vec-cache" # avoid names with including 'ret-opt' as this can cause collisions
    dtype: "float32"
```

## Running a study

To run a study simple pass the path to config, redis_url, and corpus processing function to the `run_bayes_study` function and the package will take care of the rest. 

In [3]:
# run study

# add root redis_retrieval_optimizer to path until available on pypi
import sys
import os

# Get the current notebook directory
current_dir = os.path.dirname(os.path.abspath(''))

# Go up two directory levels (adjust the number as needed)
parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add the parent directory to the Python path if it's not already there
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    print(f"Added {parent_dir} to Python path")


import os
from redis_retrieval_optimizer.bayes_study import run_bayes_study
from redis_retrieval_optimizer.corpus_processors import eval_beir
from dotenv import load_dotenv

# load environment variables containing necessary credentials
load_dotenv()

redis_url = os.environ.get("REDIS_URL", "redis://localhost:6379/0")

metrics = run_bayes_study(
    config_path="bayes_study_config.yaml",
    redis_url=redis_url,
    corpus_processor=eval_beir.process_corpus
)

[I 2025-05-22 13:19:51,785] A new study created in memory with name: test


13:19:51 redisvl.index.index INFO   Index already exists, overwriting.
13:19:52 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
13:19:52 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.69it/s]

13:19:55 root INFO   Recreating index...



Batches: 100%|██████████| 1/1 [00:00<00:00, 10.00it/s]


...

Batches: 100%|██████████| 1/1 [00:00<00:00, 49.25it/s]

13:21:08 root INFO   Recreating index...


13:21:09 root INFO   Corpus size: 3633
13:21:12 root INFO   Data indexed total_indexing_time=2.773s, num_docs=3633
13:21:13 root INFO   Saving metrics for study: 789c70ba-3208-44b1-9c06-db1888162d55, METRICS={'search_method': ['hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'vector', 'vector', 'hybrid', 'vector', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'vector', 'vector', 'vector', 'hybrid', 'hybrid', 'vector'], 'total_indexing_time': [3.559, 3.361, 3.126, 3.094, 3.443, 2.742, 4.808, 2.471, 1.896, 4.804, 3.744, 3.12, 3.262, 2.496, 3.038, 2.79, 3.907, 1.929, 3.87, 2.773], 'avg_query_time': [0.004628060402884941, 0.004614599718028916, 0.00321132922689243, 0.004435610106854985, 0.0029207316714543676, 0.001332445410383006, 0.0019653323265052063, 0.0032455301137162434, 0.001478021727995976, 0.004497922980010325, 0.00508083795246325, 0.004401772383934942, 0.0034607335140830592, 0.004710664690094467, 0.0023458756898578847, 0.001116048440844651, 0.001405441355040937, 0.003386755107

[I 2025-05-22 13:21:13,076] Trial 9 finished with value: 2.773 and parameters: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'vector', 'algorithm': 'hnsw', 'var_dtype': 'float32', 'distance_metric': 'cosine', 'ret_k': 4, 'ef_runtime': 30, 'ef_construction': 200, 'm': 64}. Best is trial 6 with value: 3.907.


13:21:13 root INFO   Completed Bayesian optimization... 


13:21:13 root INFO   Best Configuration: 6: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'vector', 'algorithm': 'hnsw', 'var_dtype': 'float16', 'distance_metric': 'cosine', 'ret_k': 8, 'ef_runtime': 30, 'ef_construction': 150, 'm': 64}:


13:21:13 root INFO   Best Score: [3.907]




In [6]:
metrics[["search_method", "algorithm", "vector_data_type", "ef_construction", "ef_runtime", "m", "avg_query_time", "total_indexing_time", "f1@k"]].sort_values(by="f1@k", ascending=False)

,search_method,algorithm,vector_data_type,ef_construction,ef_runtime,m,avg_query_time,total_indexing_time,f1@k
0,hybrid,hnsw,float16,200,50,8,0.004628,3.559,0.130712
9,hybrid,hnsw,float16,200,50,64,0.004498,4.804,0.130712
18,hybrid,hnsw,float16,150,50,16,0.004520,3.870,0.130712
17,hybrid,hnsw,float32,100,50,64,0.003387,1.929,0.130712
13,hybrid,hnsw,float16,100,10,8,0.004711,2.496,0.130712
12,hybrid,hnsw,float32,300,30,16,0.003461,3.262,0.130712
11,hybrid,hnsw,float16,100,10,16,0.004402,3.120,0.130712
1,hybrid,hnsw,float16,200,30,8,0.004615,3.361,0.130712
10,hybrid,hnsw,float16,250,10,8,0.005081,3.744,0.130712
7,hybrid,hnsw,float32,150,50,16,0.003246,2.471,0.130712
